# Create IPyParallel Profile

Borrowed from https://gist.github.com/basnijholt/c375ea2d1df6702492b619e0873d6c7c

And https://docs.bodo.ai/latest/source/ipyparallel.html

Overtime this will be automated and provide better slurm submission scripts

The two main parts to edit here are "mem" and "cpus-per-task" in the last cell. These values will depend on the current structure of your HPC cluster. One approach is to have ipython worker use an entire node. Note that because of the way cloud providers measure memory, it helps to make the worker have 1GB less than the stated memory from the cloud provider.

For example, a 8CPU 32GB instance we would set:
```
#SBATCH --mem=31G
#SBATCH --cpus-per-task=8
```

In [ ]:
! ipython profile create --parallel --profile=slurm

In [ ]:
%%writefile ~/.ipython/profile_slurm/ipcontroller_config.py

c.HubFactory.ip = u'*'
c.HubFactory.registration_timeout = 600

In [ ]:
%%writefile ~/.ipython/profile_slurm/ipengine_config.py

c.IPEngineApp.wait_for_url_file = 300
c.EngineFactory.timeout = 300

In [ ]:
#HOSTFILE=$(pwd)/hostfile.$SLURM_JOB_ID
#scontrol show hostnames > $HOSTFILE

In [ ]:
%%writefile ~/.ipython/profile_slurm/ipcluster_config.py

c.IPClusterStart.controller_launcher_class = 'SlurmControllerLauncher'
c.IPClusterEngines.engine_launcher_class = 'SlurmEngineSetLauncher'

# this is based on 4GB per core
c.SlurmEngineSetLauncher.batch_template = """#!/bin/sh
#SBATCH --ntasks={n}
#SBATCH --job-name=ipy-engine-
#SBATCH --mem=4G
#SBATCH --cpus-per-task=1

HOSTFILE=$(pwd)/hostfile.$SLURM_JOB_ID
scontrol show hostnames > $HOSTFILE

mpiexec -n {n} -machinefile $HOSTFILE python -m ipyparallel.engine --mpi --profile-dir ~/.ipython/profile_slurm --cluster-id ''
"""